In [ ]:
# %pip install folium

In [ ]:
import folium
import pandas as pd
import geopandas as gpd

In [26]:
def preprocess_data(tiktok_file, twitter_file, geojson_url):
    """Data loading and preprocessing"""
    # Load data
    tik_toks = pd.read_excel(tiktok_file)
    tweets = pd.read_excel(twitter_file)
    map_data = gpd.read_file(geojson_url)
    
    # Normalize city names 
    tik_toks['city'] = tik_toks['city'].str.lower().replace({
        'santo domingo': 'santo domingo de los tsachilas',
        'manabí': 'manabi',
        'los ríos': 'los rios'
    })

    # Process comment counts
    tik_toks['comments_count'] = tik_toks['comments_text'].apply(
        lambda x: len(str(x).split('|')) if pd.notna(x) else 0
    )
    tweets['comments_count'] = tweets['text'].apply(
        lambda x: len(str(x).split('|')) if pd.notna(x) else 0
    )

    # Convert timestamps if present
    if 'created_at' in map_data.columns:
        map_data['created_at'] = map_data['created_at'].astype(str)
    if 'updated_at' in map_data.columns:  
        map_data['updated_at'] = map_data['updated_at'].astype(str)
        
    return tik_toks, tweets, map_data

def process_social_stats(tik_toks, tweets):
    """Process social media statistics"""
    # TikTok stats
    tiktok_stats = tik_toks.groupby('city').agg({
        'comments_count': 'sum'
    }).reset_index()
    tiktok_stats['tiktok_posts_count'] = tik_toks.groupby('city').size()
    tiktok_stats.columns = ['dpa_despro', 'tiktok_comments_count', 'tiktok_posts_count']
    
    # Twitter stats 
    twitter_stats = tweets.groupby('city').agg({
        'comments_count': 'sum'
    }).reset_index()
    twitter_stats['twitter_posts_count'] = tweets.groupby('city').size()
    twitter_stats.columns = ['dpa_despro', 'twitter_comments_count', 'twitter_posts_count']
    
    return tiktok_stats, twitter_stats

def create_base_map():
    """Create base Folium map"""
    return folium.Map(
        location=[-1.8312, -78.1843],
        zoom_start=6,
        tiles='CartoDB positron'
    )

def add_choropleth_layers(m, map_data):
    """Add choropleth layers with counts"""
    metrics = {
        'tiktok_comments_count': {'color': 'Blues', 'name': 'TikTok Comments'},
        'twitter_comments_count': {'color': 'Reds', 'name': 'Twitter Comments'},
        'tiktok_posts_count': {'color': 'Greens', 'name': 'TikTok Posts'},
        'twitter_posts_count': {'color': 'Purples', 'name': 'Twitter Posts'}
    }
    
    for metric, config in metrics.items():
        choropleth = folium.Choropleth(
            geo_data=map_data.__geo_interface__,
            name=config['name'],
            data=map_data,
            columns=['dpa_despro', metric],
            key_on='feature.properties.dpa_despro',
            fill_color=config['color'],
            fill_opacity=0.7,
            line_opacity=0.2,
            legend_name=f"Number of {config['name']}"
        ).add_to(m)
        
        # Add count markers
        for _, row in map_data.iterrows():
            if row[metric] > 0:
                folium.Marker(
                    location=[row.geometry.centroid.y, row.geometry.centroid.x],
                    icon=folium.DivIcon(
                        html=f'<div style="font-size: 10pt; background: white; border: 1px solid black; border-radius: 3px; padding: 1px">{int(row[metric])}</div>'
                    )
                ).add_to(m)
    
    folium.LayerControl().add_to(m)
    return m

def create_ecuador_map(tiktok_file, twitter_file, geojson_url):
    """Main function to create the map"""
    # Load and process data
    tik_toks, tweets, map_data = preprocess_data(tiktok_file, twitter_file, geojson_url)
    
    # Process social media stats
    tiktok_stats, twitter_stats = process_social_stats(tik_toks, tweets)
    
    # Merge all data
    map_data = map_data.merge(tiktok_stats, on='dpa_despro', how='left')
    map_data = map_data.merge(twitter_stats, on='dpa_despro', how='left')
    map_data = map_data.fillna(0)
    
    # Create map
    m = create_base_map()
    m = add_choropleth_layers(m, map_data)
    
    return m

In [27]:
# Create map
map_obj = create_ecuador_map(
    "data/append_tik_tok.xlsx",
    "data/append_tweets.xlsx", 
    "https://raw.githubusercontent.com/jpmarindiaz/geo-collection/master/ecu/ecuador.geojson"
)

map_obj.save('ecuador_social_media_map.html')